In [0]:
%sql
CREATE DATABASE IF NOT EXISTS db_bronze;

In [0]:
%sql
DROP TABLE IF EXISTS db_bronze.hired_employees;
CREATE TABLE db_bronze.hired_employees(
  id bigint,
  name string,
  datetime string,
  department_id bigint,
  job_id bigint
);

DROP TABLE IF EXISTS db_bronze.departments;
CREATE TABLE db_bronze.departments(
  id bigint,
  department string
);

DROP TABLE IF EXISTS db_bronze.jobs;
CREATE TABLE db_bronze.jobs(
  id bigint,
  job string
)


In [0]:
%python
# coding: utf-8

# ||********************************************************************************************************
# || PROYECTO   		: POC -CHALLENGE GLOBLANT 
# || NOMBRE     		: challenge.py
# || TABLA DESTINO	: db_bronze.hired_employees
# ||                  db_bronze.departments 
# ||                  db_bronze.jobs
# || TABLAS FUENTES	: departments.csv
# ||                  hired_employees.csv
# ||                  jobs.csv
# || OBJETIVO   		: ETL - big data migrati
# || TIPO       		: pyspark
# || REPROCESABLE	  : NA
# || SCHEDULER		  : NA
# || JOB  		      : NA
# || VERSION   DESARROLLADOR           FECHA        DESCRIPCION
# || 1.1       ALEXIS DAVILA        21/03/23     Creacion del proceso
# *************************************************************************************************************

###
 # @section Import
 ##
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,DoubleType,FloatType,LongType

###
 # @section configuracion de recursos
 ##
spark = SparkSession.builder.appName("CSV to Database").getOrCreate()

###
 # @section funciones
 ##


def insert_csv_to_db(csv_file,table_name,nombre_columnas):
    """"
    Esta función inserta data de archivos csv a una base de datos especificando las columnas.
    :param csv_file: archivos csv
    :param table_name: nombre y esquema de la tabla en donde se inserta la data
    :param nombre_columnas: la lista de columnas de la tabla
    """
    # Lee la tabla y guarda su contenido en un DataFrame
    data_test = pd.read_csv(csv_file,sep=',', header='infer')
    df=spark.createDataFrame(data_test)
    nombres_columnas_final = nombre_columnas
    #Asigna las columnas en el dataframe
    df = df.toDF(*nombres_columnas_final)
    department_id='department_id'
    if department_id in nombres_columnas_final:
        df = df.withColumn("department_id",col("department_id").cast(LongType()))
        df = df.withColumn("job_id",col("job_id").cast(LongType()))
    else:
        print('Argumento "valida id" no válido')
    # Guarda el DataFrame en formato delta en la la tabla especificada
    df.show()
    df.write.format("delta").mode("append").option("batchsize", "1000").option("mergeSchema", "true").saveAsTable(table_name)

def backup_table(table_name, backup_path, backup_format="avro"):
    """
    Esta función crea una copia de seguridad de una tabla en formato AVRO y la guarda en el sistema de archivos.
    :param table_name: nombre de la tabla a respaldar
    :param backup_path: ruta donde se guardará el archivo de respaldo
    :param backup_format: formato en que se guardará el archivo de respaldo. Por defecto es "avro".
    """
    # Lee la tabla y guarda su contenido en un DataFrame
    df = spark.table(table_name)
    # Guarda el DataFrame en formato AVRO en la ruta especificada
    df.write.format(backup_format).save(backup_path)
    
def restore_table(table_name, backup_path, backup_format="avro"):
    """
    Esta función restaura una tabla a partir de su copia de seguridad en formato AVRO.
    :param table_name: nombre de la tabla a restaurar
    :param backup_path: ruta donde se encuentra el archivo de copia de seguridad
    :param backup_format: formato del archivo de copia de seguridad. Por defecto es "avro".
    """
    # Lee el archivo de copia de seguridad en formato AVRO y carga su contenido en un DataFrame
    df = spark.read.format(backup_format).load(backup_path)
    # Escribe el contenido del DataFrame en la tabla especificada
    df.write.mode("overwrite").saveAsTable(table_name)

def main():
    
    csv_file_1 = "https://raw.githubusercontent.com/alexis18daes/databricks_challenge_repo/dev/hired_employees.csv" 
    csv_file_2 = "https://raw.githubusercontent.com/alexis18daes/databricks_challenge_repo/dev/departments.csv"
    csv_file_3 = "https://raw.githubusercontent.com/alexis18daes/databricks_challenge_repo/dev/jobs.csv"
    
    table_name_1 = "db_bronze.hired_employees"
    table_name_2 = "db_bronze.departments"
    table_name_3 = "db_bronze.jobs"
    
    nombre_columnas_1 = ["id","name","datetime","department_id","job_id"]
    nombre_columnas_2 = ["id","name","department"]
    nombre_columnas_3 = ["id","job"]
    
    insert_csv_to_db(csv_file_1,table_name_1,nombre_columnas_1)
    insert_csv_to_db(csv_file_2,table_name_2,nombre_columnas_2)
    insert_csv_to_db(csv_file_3,table_name_3,nombre_columnas_3)
    
#Ejecucion
main()

spark.stop()

#Salida
exit()



+---+------------------+--------------------+-------------+------+
| id|              name|            datetime|department_id|job_id|
+---+------------------+--------------------+-------------+------+
|  2|          Ty Hofer|2021-05-30T05:43:46Z|            8|  null|
|  3|       Lyman Hadye|2021-09-01T23:27:38Z|            5|    52|
|  4|     Lotti Crowthe|2021-10-01T13:04:21Z|           12|    71|
|  5|    Gretna Lording|2021-10-10T22:22:17Z|            6|    80|
|  6|    Marlow Antecki|2021-04-23T23:45:42Z|            6|    95|
|  7|      Joan Rillett|2021-10-10T01:33:31Z|            9|    78|
|  8|  Ulrick Nucciotti|2021-07-24T01:28:40Z|            8|   169|
|  9|Lucretia Northcote|2021-04-01T21:22:47Z|            9|     8|
| 10|      Arty Giacobo|2022-02-08T12:27:07Z|            6|    62|
| 11|      Libbi Dowtry|2021-07-05T04:55:10Z|            6|    41|
| 12|      Jacky Oldred|2021-06-20T11:59:41Z|            7|    29|
| 13|      Raine Mowett|2021-12-23T08:23:34Z|            8|   

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
<command-882737275823810> in <cell line: 105>()
    103 
    104 #Ejecucion
--> 105 main()
    106 
    107 spark.stop()

<command-882737275823810> in main()
     99 
    100     insert_csv_to_db(csv_file_1,table_name_1,nombre_columnas_1)
--> 101     insert_csv_to_db(csv_file_2,table_name_2,nombre_columnas_2)
    102     insert_csv_to_db(csv_file_3,table_name_3,nombre_columnas_3)
    103 

<command-882737275823810> in insert_csv_to_db(csv_file, table_name, nombre_columnas)
     49     nombres_columnas_final = nombre_columnas
     50     #Asigna las columnas en el dataframe
---> 51     df = df.toDF(*nombres_columnas_final)
     52     department_id='department_id'
     53     if department_id in nombres_columnas_final:

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.